# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Import CSV file and show data
weather_data_file = "../cities_weather.csv"
weather_data_df = pd.read_csv(weather_data_file)



weather_data_df.head()


,Unnamed: 0,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Arraial do Cabo,-22.9661,-42.0278,73.36,60,75,9.22,BR,1633712355
1,1,Kargasok,59.0556,80.8572,40.89,85,88,13.56,RU,1633712469
2,2,Qaanaaq,77.4840,-69.3632,23.83,71,95,6.55,GL,1633712470
3,3,Tasiilaq,65.6145,-37.6368,34.03,50,81,11.01,GL,1633712362
4,4,Puerto Ayora,-0.7393,-90.3518,71.55,88,99,8.99,EC,1633712473


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Store locations and humidity
locations = weather_data_df[["Lat", "Lon"]].astype(float)



humidity = weather_data_df["Humidity"].astype(float)





In [5]:
#Create heatmap with heat layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                  dissipating=False, max_intensity=100,
                                  point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
weather_data_df[weather_data_df.isna().any(axis=1)]

,Unnamed: 0,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
43,43,Lüderitz,-26.6481,15.1594,60.80,64,0,13.96,NaN,1633712270
356,356,Henties Bay,-22.1160,14.2845,57.85,82,1,10.98,NaN,1633713028
478,478,Eenhana,-17.4667,16.3333,88.65,15,73,5.03,NaN,1633713214
539,539,Opuwo,-18.0607,13.8400,87.69,12,6,11.88,NaN,1633713312


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Find ideal weather conditions and show data

weather_data_df = weather_data_df.loc[(weather_data_df["Max Temp"] >= 70) & 
                                       (weather_data_df["Max Temp"] <=80) &
                                       (weather_data_df["Wind Speed"] < 10) &
                                       (weather_data_df["Cloudiness"] == 0)]

weather_data_df
                                       
                               

,Unnamed: 0,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,Safita,34.8205,36.1153,70.84,38,0,1.99,SY,1633712539
279,279,Kāzerūn,29.6195,51.6541,78.24,14,0,2.48,IR,1633712902
282,282,Tūkrah,32.5341,20.5791,75.43,71,0,4.88,LY,1633712907
311,311,Alcúdia,39.8532,3.1214,72.19,59,0,7.83,ES,1633712957
325,325,Al Bardīyah,31.7561,25.0865,74.46,57,0,7.96,LY,1633712978
481,481,Kifrī,34.6959,44.9661,79.72,16,0,9.24,IQ,1633713219
484,484,Cuamba,-14.8031,36.5372,77.59,22,0,5.03,MZ,1633713223
507,507,Tartous,34.8890,35.8866,74.50,51,0,6.40,SY,1633713258
510,510,Kanye,-24.9667,25.3327,76.10,19,0,8.59,BW,1633713263


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#Create data frame including new column for Hotel Name

weather_data_df['Hotel Name'] = ""
hotel_df = weather_data_df[["City Name", "Lat", "Lon", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date", "Hotel Name"]]
hotel_df

,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
47,Safita,34.8205,36.1153,70.84,38,0,1.99,SY,1633712539,
279,Kāzerūn,29.6195,51.6541,78.24,14,0,2.48,IR,1633712902,
282,Tūkrah,32.5341,20.5791,75.43,71,0,4.88,LY,1633712907,
311,Alcúdia,39.8532,3.1214,72.19,59,0,7.83,ES,1633712957,
325,Al Bardīyah,31.7561,25.0865,74.46,57,0,7.96,LY,1633712978,
481,Kifrī,34.6959,44.9661,79.72,16,0,9.24,IQ,1633713219,
484,Cuamba,-14.8031,36.5372,77.59,22,0,5.03,MZ,1633713223,
507,Tartous,34.8890,35.8866,74.50,51,0,6.40,SY,1633713258,
510,Kanye,-24.9667,25.3327,76.10,19,0,8.59,BW,1633713263,


In [15]:
#Loop through the dataframe and request first hotel name in the city

params = {"radius": 5000,
          "types": "lodging",
          "keyword": "hotel",
          "key": g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lon"]
    
    params["location"] = f"{lat},{lon}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    
    #print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    
hotel_df
    




,City Name,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
47,Safita,34.8205,36.1153,70.84,38,0,1.99,SY,1633712539,Safita Cham Palace
279,Kāzerūn,29.6195,51.6541,78.24,14,0,2.48,IR,1633712902,Bishapour Hotel
282,Tūkrah,32.5341,20.5791,75.43,71,0,4.88,LY,1633712907,مصيف مرحبا للعائلات
311,Alcúdia,39.8532,3.1214,72.19,59,0,7.83,ES,1633712957,Hotel Rei del Mediterrani Palace - Adults Only
325,Al Bardīyah,31.7561,25.0865,74.46,57,0,7.96,LY,1633712978,Hotel Burdy
481,Kifrī,34.6959,44.9661,79.72,16,0,9.24,IQ,1633713219,hotel
484,Cuamba,-14.8031,36.5372,77.59,22,0,5.03,MZ,1633713223,Daania Residence
507,Tartous,34.8890,35.8866,74.50,51,0,6.40,SY,1633713258,Shahin Tower Hotel
510,Kanye,-24.9667,25.3327,76.10,19,0,8.59,BW,1633713263,"Kgogomodumo Lodge, Kanye"


In [28]:
#Put hotel name into a list
hotels = hotel_df["Hotel Name"].tolist()

In [29]:
#Configure gmaps for ideal weather conditions dataframe
gmaps.configure(api_key=g_key)

In [30]:
#Store locations and humidity
locations = hotel_df[["Lat", "Lon"]].astype(float)


humidity = hotel_df["Humidity"].astype(float)

In [31]:
#Configure heat map with heat layer and hotel layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                  dissipating=False, max_intensity=100,
                                  point_radius = 1)



hotel_layer = gmaps.symbol_layer(locations, fill_color = 'rgba(0, 150, 0, 0.4)', stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
                                 info_box_content=[f"Hotel Name: {hotel}" for hotel in hotels])

fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]